In [1]:
import pickle as pkl
import logging
import numpy as np
import gensim

In [2]:
# load cleaned corpus
with open('data/cleaned_corpus.pkl', 'rb') as f:
    corpus = pkl.load(f)
with open("data/id2word.pkl", 'rb') as f:
    id2word= pkl.load(f)

In [3]:
#parameters of topic size search
max_topics = 50
min_topics = 2
step_size = 1

In [4]:
# init storage
list_topics = []
list_coherence = []
perplexity = np.full(max_topics, np.nan)
coherence = np.full(max_topics, np.nan)

In [ ]:
# loop through topic sizes
for topics in range(min_topics, max_topics, step_size):

    # enable logging
    logging.basicConfig(filename=f'log_files/gensim_{topics}_topics.log',
                    format="%(asctime)s:%(levelname)s:%(message)s",
                    level=logging.INFO)

    # Build LDA model with this number of topics
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=topics, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=1000,
                                           passes=100,
#                                            iterations=5000,
                                           alpha='auto',
                                           eval_every=1,
#                                            minimum_probability=0,
                                           per_word_topics=True)
    
    #Compute Perplexity
    perplexity[topics] = lda_model.log_perplexity(corpus)  # a measure of how good the model is. lower the better.

    # Compute Coherence Score
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    coherence[topics] = coherence_model_lda.get_coherence()
    
    #save results
    lda_model.save(f"trained_models/trained_lda_model_search_{topics}")
    np.save(f"data/perplexity_search_{topics}",perplexity)
    np.save(f"data/coherence_search_{topics}",coherence)
    
    # screen report
    print(f"Num Topics = {topics}: Perplexity = {perplexity[topics]}, Coherence = {coherence[topics]}")
    list_topics.append(topics)
    list_coherence.append(coherence[topics])
